
<h1 style='font-weight:bold;text-align:center;'>UTS Pengantar Kecerdasan Buatan</h1>

<h2 style='text-align:center;font-weight:bold'> <font color='grene'>SOAL</font></h2>
<p style='text-align:justify'> Seorang petani ingin menyeberangkan seekor kambing, seekor serigala, sayuran dengan menggunakan sebuah perahu melalui sungai. Perahu hanya bisa memuat petani & satu penumpang yang lain (kambing, serigala, atau sayuran). Jika ditinggalkan petani tersebut, maka sayuran dimakan kambing, dan kambing akan dimakan serigala.

Buatlah langkah-langkah pencarian solusi atas permasalahan PETANI, KAMBING, SERIGALA, SAYURAN, PERAHU seperti contoh kasus ember di atas sehingga semuanya dapat menyeberang sungai lengkap dengan program dalam bahasa pemrograman python!</p>




<h1> Langkah 1 := Install Modules yang diperlukan</h1>

<h2> Modules yang diperlukan adalah </h2>
<ol style='text-align:justify'>
<li>Pandas (Untuk menghasilkan dataframe dan mempermudah membaca tampilan akhir) </li>
<li>Tabulate (Untuk membantu Pandas dalam memberikan tampilan yang menarik)</li>
<li>Numpy (Pada program ini sebenarnya tidak ada menggunaka nunmpy, kita install untuk jaga-jaga apabila menginginkan kecepatan yang lebih efisien di kemudian hari)</li>
</ol>

In [20]:
# !pip install pandas
# !pip install numpy
# !pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h1> Langkah 2 := Menentukan Start state dan Final state</h1>

<h3> Penjelasan tiap baris kode akan disisipkan pada kode</h3>

In [21]:
import pandas as pd #import module pandas

pd.set_option('display.max_columns', None) #Setting agar pandas menampilkan semua kolom saat dipanggil

START_STATE = {'petani': 'asal', 'kambing': 'asal', 'serigala': 'asal', 'sayuran': 'asal', 'perahu':'asal'} #Start state default (semua berada di awal)
FINAL_STATE = {'petani': 'tujuan', 'kambing': 'tujuan', 'serigala': 'tujuan', 'sayuran': 'tujuan', 'perahu':'tujuan'} #Final state (semua berada di final)

<h1> Langkah 3 := Mengambil user input</h1>

In [22]:
user_input = {} #inisiasi sebuah dictionary untuk menyimpan user_input
valid_input = ['0','1','asal','tujuan'] #list masukan yang valid

print("""

    ========================================
    | Program nyebrang                     |
    ========================================
    Input lokasi awal dengan :
    0/asal = di sisi awal
    1/tujuan = di sisi tujuan

""")

for i in (START_STATE.keys()): #untuk tiap key yang ada pada dictionary (petani, serigala itu keys)
    while True:
        try: #coba ambil input dari user
            get_input = input(f'Input lokasi {i.capitalize()}: ') #masukan input ke variabel get_input
            if get_input in valid_input: #kalau input yang dimasukan valid (ada di list valid_input)
                user_input[i] = valid_input[valid_input.index(get_input)%2+2] #user input dengan keys (i) diisi nilainya dengan asal / tujuan, jika user input 0/1 maka akan ter konversi ke asal/tujuan juga.
                break #break While True jika input yang dimasukkan benar/valid
            else:
                raise ValueError #jika input tidak valid, maka keluarkan error
        except ValueError: #jika terjadi error
            print('Input harus asal atau tujuan')

if(user_input['perahu'] != user_input['petani']): #jika lokasi petani dan perahu berbeda
    print('Perahu harus berada di sisi yang sama dengan petani')

START_STATE = dict(START_STATE, **user_input) #timpa dictionary START_STATE dengan user_input
print(START_STATE)



    | Program nyebrang                     |
    Input lokasi awal dengan :
    0/asal = di sisi awal
    1/tujuan = di sisi tujuan


Input lokasi Petani: 0
Input lokasi Kambing: 0
Input lokasi Serigala: 0
Input lokasi Sayuran: 0
Input lokasi Perahu: 0
{'petani': 'asal', 'kambing': 'asal', 'serigala': 'asal', 'sayuran': 'asal', 'perahu': 'asal'}


<h1> Langkah 4 := Menetapkan Aturan yang berlaku</h1>

In [23]:
ATURAN = [
    {'petani': 'tujuan', 'kambing': 'tujuan', 'perahu' : 'tujuan'},  # 1. petani membawa kambing ke tujuan
    {'petani': 'tujuan', 'sayuran': 'tujuan', 'perahu' : 'tujuan'},  # 2. petani membawa sayuran ke tujuan
    {'petani': 'tujuan', 'serigala': 'tujuan','perahu' : 'tujuan'},  # 3. petani membawa serigala ke tujuan
    {'petani': 'asal', 'kambing': 'asal','perahu' : 'asal'},  # 4. petani membawa kambing ke asal
    {'petani': 'asal', 'sayuran': 'asal','perahu' : 'asal'},  # 5. petani membawa sayuran ke asal
    {'petani': 'asal', 'serigala': 'asal','perahu' : 'asal'},  # 6. petani membawa serigala ke asal
    {'petani': 'asal','perahu' : 'asal'},  # 7. petani kembali ke asal
    {'petani': 'tujuan','perahu' : 'tujuan'}  # 8. petani jalan ke tujuan
]

<h1> Langkah 5 := Membuat fungsi untuk validasi aturan yang akan digunakan</h1>

In [24]:
def is_valid(current_state:dict) -> bool:
    #Cek apakah serigala dan kambing berada di sisi yang sama
    if current_state['serigala'] == current_state['kambing'] and current_state['petani'] != current_state['serigala']:
        return False
    #Cek apakah kambing dan sayuran berada di sisi yang sama
    if current_state['kambing'] == current_state['sayuran'] and current_state['petani'] != current_state['kambing']:
        return False
    return True

def petani_check(current_state:dict, rule:dict) -> bool:
    if 'petani' in current_state.keys():
        if current_state['petani'] != rule['petani']: #jika petani tidak berada pada tempat yang sama setelah aturan dijalankan
            return True #True berarti bahwa aturan ini valid untuk dijalankan
    return False

<h1> Langkah 6 := Membuat fungsi DFS(Depth-First Search) untuk mencari hasil</h1>

In [25]:
def dfs(state:dict | None = START_STATE, visited:set | None = set()) -> list | None: #setting nilai default, nilai default dari state adalah START_STATE jika kosong, visited adalah set() jika kosong. Dengan output function berupa list atau NoneValue
    stack = [(state, [])] #inisiasi stack untuk melakukan DFS
    while stack: #jika masih ada state yang bisa dicoba dalam stack
        current_state, path = stack.pop() #nilai yang ada pada stack adalah sebuah tuple dengan 2 isi yaitu state dan list (digunakan untuk menyimpan path) yang akan dibagi ke 2 variabel current_state dan path
        if current_state == FINAL_STATE: #jika state sekarang adalah final_state
            return path + [current_state] #output path + final_state
        visited.add(tuple(current_state.items())) #tambahkan state sekarang kedalam rute yang sudah pernah dilewati agar tidak terjadi infinite loop
        for rule in ATURAN: #untuk setiap aturan
            if(petani_check(current_state,rule) and is_valid(new_state:=dict(current_state, **rule))): #cek apakah petani sekarang dan petani selanjutnya berada di tempat yang berbeda DAN langkah selanjutnya itu valid.
                if tuple(new_state.items()) not in visited: #jika sudah valid, cek apakah sudah pernah dilewati, jika belum, tambahkan ke stack.
                    stack.append((new_state, path + [current_state])) #tambahkan ke stack
    return None

<h1> Langkah 7 := Tampilkan hasil</h1>

In [26]:
solution = dfs() #memanggil fungsi dfs dengan parameter default

In [27]:
if solution != None: #jika ada solusi
    df = pd.DataFrame(columns=list(START_STATE.keys())) #buat dataframe dengan kolom berupa keys pada START_STATE
    for i in solution: #untuk setiap state (path) pada solusi
        df.loc[len(df.index)] = i #tambahkan ke dataframe
    df.rename(str.capitalize, axis='columns', inplace=True) #Ubah kolom dataframe menjadi kapital
    print(df.to_markdown()) #tampilkan
else:
    print('Tidak ada solusi')

|    | Petani   | Kambing   | Serigala   | Sayuran   | Perahu   |
|---:|:---------|:----------|:-----------|:----------|:---------|
|  0 | asal     | asal      | asal       | asal      | asal     |
|  1 | tujuan   | tujuan    | asal       | asal      | tujuan   |
|  2 | asal     | tujuan    | asal       | asal      | asal     |
|  3 | tujuan   | tujuan    | tujuan     | asal      | tujuan   |
|  4 | asal     | asal      | tujuan     | asal      | asal     |
|  5 | tujuan   | asal      | tujuan     | tujuan    | tujuan   |
|  6 | asal     | asal      | tujuan     | tujuan    | asal     |
|  7 | tujuan   | tujuan    | tujuan     | tujuan    | tujuan   |


<h1> Langkah 8 := ATAU, bisa gunakan display(), hal ini tidak memerlukan Module Tabulate.</h1>

In [28]:
if solution != None: #jika ada solusi
    df = pd.DataFrame(columns=list(START_STATE.keys())) #buat dataframe dengan kolom berupa keys pada START_STATE
    for i in solution: #untuk setiap state (path) pada solusi
        df.loc[len(df.index)] = i #tambahkan ke dataframe
    df.rename(str.capitalize, axis='columns', inplace=True) #Ubah kolom dataframe menjadi kapital
    display(df) #tampilkan
else:
    print('Tidak ada solusi')

,Petani,Kambing,Serigala,Sayuran,Perahu
0,asal,asal,asal,asal,asal
1,tujuan,tujuan,asal,asal,tujuan
2,asal,tujuan,asal,asal,asal
3,tujuan,tujuan,tujuan,asal,tujuan
4,asal,asal,tujuan,asal,asal
5,tujuan,asal,tujuan,tujuan,tujuan
6,asal,asal,tujuan,tujuan,asal
7,tujuan,tujuan,tujuan,tujuan,tujuan
